In [137]:
import os, glob

microservices = dict()
path = "D:\\Projects\\Xtensus\\xteUtile\\JDLConfiguration"
data = dict()
for filename in glob.glob(os.path.join(path, '*.jdl')):
    startIndex = filename.rfind('\\')+1
    endIndex = filename.rfind('Config')
    microservices[filename[startIndex:endIndex].lower()] = dict()
    microservices[filename[startIndex:endIndex].lower()]["entities"] = dict()
    with open(os.path.join(os.getcwd(), filename), 'r') as file:
        data = file.readlines()
        microservices[filename[startIndex:endIndex].lower()]["data"] = data
        for line in data:
            if ("paginate" in line):
                entityIndexStart = line.find(next(filter(str.isupper, line), ''))
                entityIndexEnd = line.find("with")-1
                microservices[filename[startIndex:endIndex].lower()]["entities"][line[entityIndexStart:entityIndexEnd].strip()] = set()
print(microservices)

{'personnel': {'entities': {'Personnel': set(), 'Personnelgrade': set(), 'Personnelchauffeur': set(), 'Personnelattributionquotacarburant': set(), 'Personnelautrepersonnel': set()}, 'data': ['entity Personnel {\n', '    personnelReference Integer required\n', '    personnelReferenceClt Integer\n', '    personnelMatricule String\n', '    personnelNom String\n', '    personnelNomAR String\n', '    personnelPrenom String\n', '    personnelPrenomAR String\n', '    DateNaissance ZonedDateTime\n', '    DateEmbauche ZonedDateTime\n', '    personnelBenificiereCarburant Boolean\n', '    personnelCellulaire String\n', '    personnelNumeroTelephone String\n', '    personnelNumeroPermisConduire String\n', '    personnelDateLivraisonPermisConduire String\n', '    personnelSalaire Integer\n', '    personnelDateSortie String\n', '    personnelDateGrade Integer\n', '    fonctionId Integer unique\n', '    etatId Integer unique\n', '    uniteId Integer unique\n', '    categoriePersonnel Integer\n', '}\n

In [138]:
for microservice in microservices.keys():
    newEntity = False
    entityName = ""
    for line in microservices[microservice]["data"]:
        if ("entity" in line):
            newEntity = True
            entityName = line[line.find(next(filter(str.isupper, line), '')):line.find("{")-1].strip()
            print("Found New Entity :", entityName)
            continue
        if ("}" in line):
            newEntity = False
            entityName = ""
        if (newEntity==True):
            for entity in microservices[microservice]["entities"].keys():
                if (entity==entityName):
                    #print(entity)
                    for otherMicroservice in microservices.keys():
                        if (otherMicroservice == microservice):
                            continue
                        if (otherMicroservice in line):
                            index = -1
                            for c in line:
                                index+=1
                                if (c==' '):
                                    break
                            print(line)
                    break



Found New Entity : Personnel
Found New Entity : Personnelgrade
Found New Entity : Personnelchauffeur
Found New Entity : Personnelattributionquotacarburant
Found New Entity : Personnelautrepersonnel
Found New Entity : Boncommande
Found New Entity : Bonlivraison
Found New Entity : Fournisseur
Found New Entity : Facture
Found New Entity : ArticleMouvement
Found New Entity : Article
    vehiculeModeleMarqueId Integer unique

Found New Entity : ArticleCategory
Found New Entity : ArticleUnite
Found New Entity : DepotNiveau1
Found New Entity : DepotNiveau2
Found New Entity : DepotNiveau3
Found New Entity : Ligneboncommande
Found New Entity : Bonsortiereparationentretiendevis
Found New Entity : Bonretour
Found New Entity : Boncommandeaffectationvehicule
Found New Entity : Fodec
Found New Entity : Factureacciden
Found New Entity : Vehicule
Found New Entity : MDeafVehiculeGenre
Found New Entity : MDeafVehiculeModeleCommercial
Found New Entity : VehiculeUnite
Found New Entity : Unite
Found New En